In [29]:
# ドライブ読み込み
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/Colab Notebooks"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks


In [30]:
!pip install japanize_matplotlib

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns
import sympy
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from PIL import Image

**・Pclass**： 旅客クラス(1＝1等船室、2＝2等船室、3＝3等船室)
		裕福さの目安である。

**・Sex**： 性別(male＝男性、female＝女性)

**・Age**： 年齢。一部の乳児は小数値。

**・SibSp**： 同乗している兄弟(Siblings)や配偶者(Spouses)の数

**・Parch**： 同乗している親(Parents)や子供(Children)の数

**・Ticket**： チケット番号

**・Fare**： 旅客運賃

**・Cabin**： 客室番号

**・Embarked**： 出港地（C＝Cherbourg：シェルブール、Q＝Queenstown：クイーンズタウン、S＝Southampton：サウサンプトン）

**・Survived**：生存状況（0＝死亡、1＝生存）。通常はこの数値が目的変数として使われる。

## 作業(1) train.csvの読み込み


train.csv を読み込み、

”PassengerId”,  “Survived”,  “Ticket”

を除く他の列をデータフレーム(変数名 df )に格納。

また、”Survived”列をラベルとして NumPy配列 にして

変数 y_train_true に代入。

In [91]:
# ------------------------(1)------------------------

df = pd.read_csv("train.csv").drop(['PassengerId', 'Survived', 'Ticket'], axis=1)
print(df.head()) # 確認用

y_train_true = pd.read_csv("train.csv")["Survived"].values.reshape(-1, 1)
# print(y_train_true) # 確認用

   Pclass     Sex   Age  SibSp  Parch     Fare Cabin Embarked
0       3    male  22.0      1      0   7.2500   NaN        S
1       1  female  38.0      1      0  71.2833   C85        C
2       3  female  26.0      0      0   7.9250   NaN        S
3       1  female  35.0      1      0  53.1000  C123        S
4       3    male  35.0      0      0   8.0500   NaN        S


## 作業(2)欠損値のある列名とその数

**scikit-learn は欠損値があるとモデルを作成できない。**

よって、欠損値がどこにあるか調べる。

欠損値のある列の名前を全て挙げ、さらにその列には欠損値がいくつあるか数える。

In [92]:
# ------------------------(2)------------------------

# 各列の欠損値の数を計算
missing_values = df.isnull().sum()

print(missing_values) # 確認用

# 欠損値がある列とその数を表示
for column, count in missing_values.items():
    if count > 0:
      print(f'{column}に欠損値が{count}カ所あります')

# Age=177, Cabin=687, Embarked=2

Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Cabin       687
Embarked      2
dtype: int64
Ageに欠損値が177カ所あります
Cabinに欠損値が687カ所あります
Embarkedに欠損値が2カ所あります


## 作業(3)特定の列の削除

“Cabin”列は欠損値が多いので、df から列ごと削除

In [93]:
# ------------------------(3)------------------------

print("削除前")
print(df)

df = df.drop("Cabin", axis=1)
print("削除後")
print(df)

削除前
     Pclass     Sex   Age  SibSp  Parch     Fare Cabin Embarked
0         3    male  22.0      1      0   7.2500   NaN        S
1         1  female  38.0      1      0  71.2833   C85        C
2         3  female  26.0      0      0   7.9250   NaN        S
3         1  female  35.0      1      0  53.1000  C123        S
4         3    male  35.0      0      0   8.0500   NaN        S
..      ...     ...   ...    ...    ...      ...   ...      ...
886       2    male  27.0      0      0  13.0000   NaN        S
887       1  female  19.0      0      0  30.0000   B42        S
888       3  female   NaN      1      2  23.4500   NaN        S
889       1    male  26.0      0      0  30.0000  C148        C
890       3    male  32.0      0      0   7.7500   NaN        Q

[891 rows x 8 columns]
削除後
     Pclass     Sex   Age  SibSp  Parch     Fare Embarked
0         3    male  22.0      1      0   7.2500        S
1         1  female  38.0      1      0  71.2833        C
2         3  female  26.0 

## 作業(4)欠損値の補完

欠損値のある列に対する操作


単一代入法との指示の為、今回は平均値で補完する

AgeとEmbarkedに対して

単一代入法は、データの分散を過小評価してしまうことになるため、

Ageの欠損値には平均値を、Embarkedの欠損値には最頻値を代入する

In [94]:
# ------------------------(4)------------------------

# 平均値で補完
df['Age'] = df['Age'].fillna(df['Age'].mean(numeric_only = True))
# 最頻値で補完
Embarked_mode = df['Embarked'].mode()[0]
df['Embarked'] = df['Embarked'].fillna(Embarked_mode)

print(df) # 確認

     Pclass     Sex        Age  SibSp  Parch     Fare Embarked
0         3    male  22.000000      1      0   7.2500        S
1         1  female  38.000000      1      0  71.2833        C
2         3  female  26.000000      0      0   7.9250        S
3         1  female  35.000000      1      0  53.1000        S
4         3    male  35.000000      0      0   8.0500        S
..      ...     ...        ...    ...    ...      ...      ...
886       2    male  27.000000      0      0  13.0000        S
887       1  female  19.000000      0      0  30.0000        S
888       3  female  29.699118      1      2  23.4500        S
889       1    male  26.000000      0      0  30.0000        C
890       3    male  32.000000      0      0   7.7500        Q

[891 rows x 7 columns]


## 作業(5)ラベルエンコーディング

SexとEmbarkedはobject型なので、数値に変える

Sex： 性別(male＝男性、female＝女性)

Embarked： 出港地（C＝Cherbourg：シェルブール、Q＝Queenstown：クイーンズタウン、S＝Southampton：サウサンプトン）

In [95]:
# ------------------------(5)------------------------

print(df) # 変更前
df['Sex'] = df['Sex'].replace({'male': 0, 'female': 1})
df['Embarked'] = df['Embarked'].replace({'C': 0, 'Q': 1, 'S': 2})
print(df) # 変更後

     Pclass     Sex        Age  SibSp  Parch     Fare Embarked
0         3    male  22.000000      1      0   7.2500        S
1         1  female  38.000000      1      0  71.2833        C
2         3  female  26.000000      0      0   7.9250        S
3         1  female  35.000000      1      0  53.1000        S
4         3    male  35.000000      0      0   8.0500        S
..      ...     ...        ...    ...    ...      ...      ...
886       2    male  27.000000      0      0  13.0000        S
887       1  female  19.000000      0      0  30.0000        S
888       3  female  29.699118      1      2  23.4500        S
889       1    male  26.000000      0      0  30.0000        C
890       3    male  32.000000      0      0   7.7500        Q

[891 rows x 7 columns]
     Pclass  Sex        Age  SibSp  Parch     Fare  Embarked
0         3    0  22.000000      1      0   7.2500         2
1         1    1  38.000000      1      0  71.2833         0
2         3    1  26.000000      0   

## 作業(6)欠損値を再度確認

In [96]:
# ------------------------(6)------------------------

missing_values_2 = df.isnull().sum()

print(missing_values_2) # 確認用

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64


## 作業(7)test.csvの読み込み

test.csv を読み込み、

”PassengerId”,   “Ticket”

を除く他の列をデータフレーム(変数名 df_test )に格納

In [97]:
# ------------------------(7)------------------------

df_test = pd.read_csv("test.csv").drop(['PassengerId', 'Ticket'], axis=1)
print(df.head()) # 確認用

   Pclass  Sex   Age  SibSp  Parch     Fare  Embarked
0       3    0  22.0      1      0   7.2500         2
1       1    1  38.0      1      0  71.2833         0
2       3    1  26.0      0      0   7.9250         2
3       1    1  35.0      1      0  53.1000         2
4       3    0  35.0      0      0   8.0500         2


## 作業(8)test.cavに対して(1)～(6)の操作

In [98]:
# ------------------------(8)------------------------

# 欠損値の確認

# 各列の欠損値の数を計算
missing_values_3 = df_test.isnull().sum()

print(missing_values_3) # 確認用

# 欠損値がある列とその数を表示
for column, count in missing_values_3.items():
    if count > 0:
      print(f'{column}に欠損値が{count}カ所あります')

# Age=86, Fare=1, Cabin=327

# Cabin列削除

print("\n\n削除前")
print(df_test)

df_test = df_test.drop("Cabin", axis=1)
print("削除後")
print(df_test)


# 欠損値の補完
# AgeとFareそれぞれ平均値の単一代入法とする

df_test['Age'] = df_test['Age'].fillna(df_test['Age'].mean(numeric_only = True))
df_test['Fare'] = df_test['Fare'].fillna(df_test['Fare'].mean(numeric_only = True))
print("\n\n変更後")
print(df_test)


# ラベルエンコーディング
df_test['Sex'] = df_test['Sex'].replace({'male': 0, 'female': 1})
df_test['Embarked'] = df_test['Embarked'].replace({'C': 0, 'Q': 1, 'S': 2})
print(df_test) # 変更後


# 欠損値の再度確認

missing_values_4 = df_test.isnull().sum()

print(missing_values_4) # 確認用

Pclass        0
Sex           0
Age          86
SibSp         0
Parch         0
Fare          1
Cabin       327
Embarked      0
dtype: int64
Ageに欠損値が86カ所あります
Fareに欠損値が1カ所あります
Cabinに欠損値が327カ所あります


削除前
     Pclass     Sex   Age  SibSp  Parch      Fare Cabin Embarked
0         3    male  34.5      0      0    7.8292   NaN        Q
1         3  female  47.0      1      0    7.0000   NaN        S
2         2    male  62.0      0      0    9.6875   NaN        Q
3         3    male  27.0      0      0    8.6625   NaN        S
4         3  female  22.0      1      1   12.2875   NaN        S
..      ...     ...   ...    ...    ...       ...   ...      ...
413       3    male   NaN      0      0    8.0500   NaN        S
414       1  female  39.0      0      0  108.9000  C105        C
415       3    male  38.5      0      0    7.2500   NaN        S
416       3    male   NaN      0      0    8.0500   NaN        S
417       3    male   NaN      1      1   22.3583   NaN        C

[418 rows x 8 colu

## 作業(9)モデル作成と学習データに対する予測

df と教師データ y_train_true を使って、scikit-learn の SVC クラスを使って

生存状況を予測するモデルを作成

なお、学習の際は df の全てのデータを使う(train_test_split しない)

また、作ったモデルの学習データの予測値の正解率を求める

In [99]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# SVCインスタンス
model = SVC()
# モデルを学習
model.fit(df, y_train_true)

# 学習データに対する予測
y_pred = model.predict(df)

# 正解率を計算
accuracy = accuracy_score(y_train_true, y_pred)
print(f"\n\n作ったモデルの学習データの予測値の正解率: {accuracy}")



作ったモデルの学習データの予測値の正解率: 0.6857463524130191


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## 作業(10)テストデータに対する予測

gender_submission.csv という test.csv の人々の生存状況のデータある

これを用いて、作業(9)で作成したモデルの test.csv の予測値の正解率を求める

In [100]:
# gender_submission.csvの確認

df_gs = pd.read_csv("gender_submission.csv")
print(df_gs)

# PassengerIdを削除する
df_gs = df_gs.drop("PassengerId", axis=1)
print("\n\n削除後")
print(df_gs)

# numpy配列にしてy_test_trueとする
y_test_true = df_gs["Survived"].values.reshape(-1, 1)

# テストデータに対する予測
y_test_pred = model.predict(df_test)

# 正解率を計算
accuracy_test = accuracy_score(y_test_true, y_test_pred)
print(f"\n\n作成したモデルの test.csv の予測値の正解率: {accuracy_test}")

     PassengerId  Survived
0            892         0
1            893         1
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]


削除後
     Survived
0           0
1           1
2           0
3           0
4           1
..        ...
413         0
414         1
415         0
416         0
417         0

[418 rows x 1 columns]


作成したモデルの test.csv の予測値の正解率: 0.6555023923444976


## おまけ　標準化を使うと正解率が上がるかどうか

### 結果

##### 作ったモデルの学習データの予測値の正解率

0.6857463524130191 → 0.8383838383838383

#### 作成したモデルの test.csv の予測値の正解率

0.6555023923444976 → 0.930622009569378



In [101]:
# train.csvに対しての処理



df = pd.read_csv("train.csv").drop(['PassengerId', 'Survived', 'Ticket'], axis=1)
# print(df.head()) # 確認用

y_train_true = pd.read_csv("train.csv")["Survived"].values.reshape(-1, 1)
# print(y_train_true) # 確認用

# 各列の欠損値の数を計算
missing_values = df.isnull().sum()

# print(missing_values) # 確認用

# 欠損値がある列とその数を表示
# for column, count in missing_values.items():
#     if count > 0:
#       print(f'{column}に欠損値が{count}カ所あります')

# Age=177, Cabin=687, Embarked=2

# print("削除前")
# print(df)

df = df.drop("Cabin", axis=1)
# print("削除後")
# print(df)

# 平均値で補完
df['Age'] = df['Age'].fillna(df['Age'].mean(numeric_only = True))
# 最頻値で補完
Embarked_mode = df['Embarked'].mode()[0]
df['Embarked'] = df['Embarked'].fillna(Embarked_mode)

# print(df) # 確認

# print(df) # 変更前
df['Sex'] = df['Sex'].replace({'male': 0, 'female': 1})
df['Embarked'] = df['Embarked'].replace({'C': 0, 'Q': 1, 'S': 2})
# print(df) # 変更後

missing_values_2 = df.isnull().sum()

# print(missing_values_2) # 確認用



# test.csvに対しての処理



df_test = pd.read_csv("test.csv").drop(['PassengerId', 'Ticket'], axis=1)
# print(df.head()) # 確認用

# 欠損値の確認

# 各列の欠損値の数を計算
missing_values_3 = df_test.isnull().sum()

# print(missing_values_3) # 確認用

# 欠損値がある列とその数を表示
# for column, count in missing_values_3.items():
#     if count > 0:
#       print(f'{column}に欠損値が{count}カ所あります')

# Age=86, Fare=1, Cabin=327

# Cabin列削除

# print("\n\n削除前")
# print(df_test)

df_test = df_test.drop("Cabin", axis=1)
# print("削除後")
# print(df_test)


# 欠損値の補完
# AgeとFareそれぞれ平均値の単一代入法とする

df_test['Age'] = df_test['Age'].fillna(df_test['Age'].mean(numeric_only = True))
df_test['Fare'] = df_test['Fare'].fillna(df_test['Fare'].mean(numeric_only = True))
# print("\n\n変更後")
# print(df_test)

# ラベルエンコーディング
df_test['Sex'] = df_test['Sex'].replace({'male': 0, 'female': 1})
df_test['Embarked'] = df_test['Embarked'].replace({'C': 0, 'Q': 1, 'S': 2})
# print(df_test) # 変更後

# 欠損値の再度確認

missing_values_4 = df_test.isnull().sum()

# print(missing_values_4) # 確認用



# ---------------------------------ここから-----------------------------------



from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# 標準化
scaler = StandardScaler()
scaler.fit(df)
df_scaled = scaler.transform(df)

# SVCインスタンス
model = SVC()
# モデルを学習
model.fit(df_scaled, y_train_true)

# 学習データに対する予測
y_pred = model.predict(df_scaled)

# 正解率を計算
accuracy = accuracy_score(y_train_true, y_pred)
print(f"\n\n作ったモデルの学習データの予測値の正解率: {accuracy}")


# gender_submission.csvの確認

df_gs = pd.read_csv("gender_submission.csv")
# print(df_gs)

# PassengerIdを削除する
df_gs = df_gs.drop("PassengerId", axis=1)
# print("\n\n削除後")
# print(df_gs)

# numpy配列にしてy_test_trueとする
y_test_true = df_gs["Survived"].values.reshape(-1, 1)

# 標準化適応
# この際、学習データで用いたscalerインスタンスを使うこと
df_test_scaled = scaler.transform(df_test)

# テストデータに対する予測
y_test_pred = model.predict(df_test_scaled)

# 正解率を計算
accuracy_test = accuracy_score(y_test_true, y_test_pred)
print(f"\n\n作成したモデルの test.csv の予測値の正解率: {accuracy_test}")



作ったモデルの学習データの予測値の正解率: 0.8383838383838383


作成したモデルの test.csv の予測値の正解率: 0.930622009569378


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
